In [2]:
import pandas as pd
from sodapy import Socrata
import pickle, io
import openpyxl
import matplotlib.pyplot as plt
import matplotlib

In [3]:
# Add codes for more cities
codis_municipis = ['430120', '439076', '430430', '430477', '430509', '430957', '430976', '431009', '431037', '431093', '431114', '431227', '431264', '431310', '431359', '439057', '431444', '431482', '431536', '431649', '431711', '431665']

In [4]:
# DON'T EXECUTE MANY TIMES: a file will be saved so no need to access the API more than once.

# https://analisi.transparenciacatalunya.cat/Habitatge/Preu-mitj-del-lloguer-d-habitatges-per-municipi/qww9-bvhh

client = Socrata("analisi.transparenciacatalunya.cat",
                 "s4YrtY4AKxmM0rFio3wIMDn8Y")

query = """
SELECT 
    `codi_territorial`, `nom_territori`, `any`, `periode`, `habitatges`, `renda` 
WHERE 
    `codi_territorial` IN ('43012','43907','43043','43047','43050','43095','43097','43100','43103','43109','43111','43122','43126','43131','43135','43905','43144','43148','43153','43164','43171','43166') 
AND
    `any` IN ('2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011')
AND
    `renda` > 0
"""

# Results returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("qww9-bvhh", query=query)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)
df.to_pickle("output/preu_lloguer.pkl")

In [5]:
# Read file obtained from API
df = pd.read_pickle("output/preu_lloguer.pkl")

# Rename columns and convert to Numbers
df.rename(columns={'codi_territorial': 'codi_municipi', 'nom_territori': 'municipi'}, inplace=True)
df["habitatges"] = pd.to_numeric(df["habitatges"])
df["any"] = pd.to_numeric(df["any"])
df["codi_municipi"] = pd.to_numeric(df["codi_municipi"])
df["renda"] = pd.to_numeric(df["renda"])

# Get longest period of each year (gener-desembre in most cases)
group = df.groupby(['codi_municipi', 'any'])
df = group.apply(lambda x: df.iloc[x['habitatges'].idxmax()])

# Save into excel file
df.to_excel("output/preu_lloguer.xlsx", index=False)

# TODO: Falta info de Nou de Gaià (3 anys), Salomó (4 anys), vespella de gaià, renau